In [8]:
# import neccessary libraries 
import tifffile
import numpy as np
from PIL import Image
import os 
from tqdm import tqdm 
from tkinter import Tcl

# My utilies 
from poreUtils import * 


from scipy import ndimage as nd

import pypore3d 
from pypore3d import *
from pypore3d.p3dFiltPy import *
from pypore3d.p3dBlobPy import *
from pypore3d.p3dSkelPy import *
from pypore3d.p3dSITKPy import *



from glob import glob 
import json 
import ctypes 

In [64]:
data_dir = 'D:\\sagar\\Data\\MD_1264_A5_1_Z0.0mm_corr_phrt\\roi\\'
selected_files = os.listdir(data_dir)

out_dir = 'D:\\sagar\\Data\\MD_1264_A5_1_Z0.0mm_corr_phrt\\result_after_removed_bad_slices\\'

In [61]:
# global min and max 
gbMin = 0.0005
gbMax = 0.003

In [65]:
selected_files

['0-300x1050-1350x2100-2400',
 '0-300x1050-1350x2900-3200',
 '0-300x1450-1750x2100-2400',
 '0-300x1450-1750x2700-3000',
 '0-300x850-1150x2900-3200',
 '1000-1300x1250-1550x2900-3200',
 '1000-1300x1250-1550x3300-3600',
 '1000-1300x650-950x1700-2000',
 '1000-1300x850-1150x1700-2000',
 '1200-1500x1250-1550x3300-3600',
 '1200-1500x850-1150x1700-2000',
 '200-500x1050-1350x2100-2400',
 '200-500x1050-1350x2900-3200',
 '200-500x1450-1750x2700-3000',
 '200-500x850-1150x2300-2600',
 '200-500x850-1150x2900-3200',
 '400-700x1050-1350x2900-3200',
 '400-700x1250-1550x2700-3000',
 '400-700x850-1150x1900-2200',
 '400-700x850-1150x2900-3200',
 '600-900x1250-1550x2900-3200',
 '600-900x850-1150x2700-3000',
 '600-900x850-1150x3100-3400',
 '800-1100x1050-1350x1700-2000',
 '800-1100x1250-1550x3100-3400',
 '800-1100x850-1150x1700-2000',
 '800-1100x850-1150x2300-2600']

In [67]:
for afile in (selected_files):
    file_path = os.path.join(data_dir, afile)
    raw_vol = []
    #sorting the slices according to their names like in windows 
    slices = Tcl().call('lsort', '-dict', os.listdir(file_path))
    for aSlice in slices:
        img = Image.open(os.path.join(file_path, aSlice))
        imgarray = np.array(img)
        print(np.median(imgarray))
        if np.isclose(np.median(imgarray), 0.0005)== True:
            print(afile, aSlice)
#         else:
#             raw_vol.append(imgarray)
        
#     raw_vol = np.asarray(raw_vol)
#     #raw_vol = np.nan_to_num(raw_vol)
#     raw_vol = np.clip(raw_vol, a_min=gbMin, a_max=gbMax)
#     raw_vol = norm8bit(raw_vol)#, minVal=gbMin, maxVal=gbMax)
#     fName = os.path.join(out_dir, afile)
#     raw_vol.tofile(fName + '_8bit.raw')
    
    

0.0009713472
0.00097401685
0.0009757038
0.0009785702
0.0009812657
0.0009839863
0.000988262
0.0009945901
0.0009993813
0.0010035262
0.0010066705
0.0010101765
0.0010132304
0.001014828
0.0010152806
0.0010158059
0.0010166373
0.0010167474
0.0010179236
0.0010183197
0.0010172919
0.0010151895
0.0010146269
0.0010146752
0.0010173065
0.0010193227
0.0010211056
0.0010198667
0.0010189445
0.0010184561
0.0010178017
0.0010154888
0.0010148063
0.0010149111
0.0010150688
0.001014264
0.0010157728
0.0010167421
0.0010173693
0.0010179568
0.0010181754
0.0010190108
0.0010190513
0.0010190975
0.0010194052
0.0010182058
0.0010158927
0.0010142047
0.001010935
0.0010087014
0.0010070542
0.0010055436
0.0010047865
0.0010048604
0.001006155
0.0010084204
0.0010102213
0.0010120738
0.001014013
0.0010167111
0.0010191756
0.0010208662
0.0010222753
0.0010274295
0.0010317407
0.0010345764
0.001036106
0.0010364447
0.0010372298
0.0010370165
0.0010369687
0.0010380441
0.0010394772
0.001041139
0.0010427409
0.0010453579
0.00104553
0.001048

0.001057175
0.0010587482
0.0010597353
0.0010598206
0.0010589916
0.0010595692
0.00106096
0.0010642488
0.0010685287
0.0010715632
0.0010730352
0.0010738766
0.0010731294
0.001073262
0.0010757075
0.0010768196
0.0010766389
0.0010747963
0.0010721688
0.0010695287
0.0010687287
0.0010675293
0.0010655094
0.0010647583
0.0010646374
0.00106524
0.0010661953
0.0010691201
0.0010738664
0.0010767758
0.0010786641
0.0010808387
0.0010817896
0.001083124
0.0010813954
0.0010789286
0.0010743395
0.0010696187
0.0010637108
0.0010599596
0.0010570835
0.0010536575
0.0010500993
0.0010481225
0.0010480683
0.0010483573
0.0010485026
0.0010480735
0.0010474977
0.0010474559
0.0010474562
0.0010487234
nan
0.0010569631
0.0010619638
0.0010676521
0.0010712075
0.0010724586
0.0010734177
0.0010726503
0.0010731502
0.0010714611
0.0010671787
0.0010630502
0.0010604328
0.0010576779
0.0010549388
0.0010526301
0.0010496217
0.0010484664
0.0010484129
0.0010491111
0.0010493298
0.0010492436
0.0010484998
0.0010458238
0.0010450822
0.0010455541
0.

0.0011307898
0.0011327895
0.0011344864
0.0011364238
0.0011369076
0.0011370276
0.0011366915
0.001137136
0.0011374969
0.0011382352
0.0011378232
0.0011388059
0.001139517
0.0011163079
0.0011416127
0.0011451328
0.0011484654
0.0011509947
0.0011541375
0.0011578358
0.0011601781
0.0011590656
0.0011570476
0.0011537613
0.0011505161
0.0011485917
0.0011454778
0.0011437756
0.0011442627
0.0011451158
0.0011470604
0.001148155
0.0011493685
0.0011503518
0.0011494863
0.0011494411
0.0011505745
0.0011526737
0.0011539969
0.0011543416
0.0011527042
0.0011505969
0.0011486276
0.0011470055
0.0011476406
0.0011479483
0.0011472658
0.0011465165
0.0011450312
0.0011435826
0.0011436499
0.0011430301
0.0011411428
0.0011375209
0.0011360715
0.0011342936
0.0011325872
0.0011336182
0.0011347509
0.0011358538
0.001137423
0.0011392742
0.0011416138
0.0011432082
0.0011461632
0.0011477619
0.0011487836
0.0011495852
0.0011496041
0.0011494667
0.0011459452
0.0011420813
0.0011373578
0.0011336559
0.0011302326
0.0011249692
0.0011220286
0.0

0.0010595305
0.0010624693
0.0010660689
0.0010682712
0.0010709361
0.0010728105
0.0010746911
0.0010739138
0.0010713707
0.0010720487
0.0010711277
0.0010696205
0.0010663739
0.0010640994
0.0010607135
0.0010601522
0.0010600153
0.0010579667
0.0010555197
0.0010559976
0.0010568921
0.0010594291
0.0010639132
0.0010713788
0.0010772934
0.0010820043
0.0010840255
0.0010848942
0.0010843513
0.0010835808
0.0010824252
0.0010839148
0.0010867068
0.0010895021
0.0010908714
0.0010915481
0.0010900856
0.0010895177
0.0010879363
0.0010893049
0.0010911496
0.0010894854
0.0010875475
0.001083402
0.0010801372
0.001079174
0.0010806404
0.0010792601
0.0010800362
0.0010809093
0.0010801802
0.0010806674
0.0010810586
0.0010812663
0.0010826685
0.0010808681
0.0010789046
0.0010792685
0.0010806269
0.0010811198
0.0010845426
0.00108869
0.0010924203
0.0010952239
0.0010964065
0.0010958639
0.0010950763
0.0010946513
0.0010927044
0.0010896622
0.0010877494
0.0010855712
0.0010835916
0.0010798182
0.0010743897
0.001071664
0.0010708431
0.00

0.00088147377
0.0008792402
0.00087608455
0.0008734208
0.0008687866
0.00086460274
0.000862463
0.00086112716
0.000860289
0.0008596784
0.00085762946
0.0008561886
0.0008551958
0.00085300475
0.00085192046
0.0008506502
0.0008496207
0.0008512479
nan
0.00085428037
0.0008558952
0.00085851894
0.00086117623
0.00086286524
0.0008641674
0.00086553016
0.00086550333
0.0008648224
0.00086349994
0.0008629792
0.0008629215
0.0008634068
0.00086461403
0.00086552196
0.0008664266
0.00086641917
0.0008658826
0.00086545636
0.00086473336
0.00086501916
0.000864673
0.00086348667
0.0008621245
0.0008624645
0.0008636315
0.0008651008
0.00086637796
0.00086954073
0.00087174965
0.0008743855
0.0008750337
0.0008753714
0.00087714544
0.0008783723
0.0008804243
0.0008811463
0.0008813597
0.000880955
0.00087893364
0.0008755597
0.0008687037
0.0008624671
0.0008584311
0.0008565645
0.00085807435
0.0008601245
0.0008621081
0.00086400035
0.00086550025
0.000865591
0.0008681454
0.0008703722
0.000871757
0.0010792685
0.0010806269
0.001081119

...

0.0010360419
0.0010339133
0.0010312211
0.0010295345
0.0010277489
0.0010266963
0.0010257087
0.0010234595
0.0010212599
0.0010198955
0.0010209973
0.0010219407
0.0010245526
0.0010272074
0.0010285071
0.0010292791
0.0010295468
0.0010309706
0.0010321569
0.0010308687
0.0010287748
0.0010292131
0.0010295869
0.0010301094
0.0010317706
0.0010329157
0.0010373287
0.0010392772
0.0010396069
0.0010395971
0.0010397407
0.001041711
0.0010445578
0.0010476725
0.0010507213
0.0010527079
0.0010519021
0.0010483776
0.0010450641
0.0010418186
0.0010388842
0.0010395251
0.0010415681
0.0010429884
0.0010438006
0.0010452453
0.0010476636
0.0010457747
0.0010427947
0.0010394063
0.0010391525
0.0010398075
0.0010424764
0.0010439104
0.00104351
0.001041352
0.0010400217
0.0010393385
0.0010387728
0.0010400825
0.0010431319
0.0010441737
0.0010456135
0.0010467195
0.0010481782
0.001047296
0.0010452427
0.0010439914
0.0010446724
0.0010431046
0.0010417654
0.0010397185
0.0010399455
0.0010399148
0.0010402197
0.0010408922
0.0010386694
0.00

0.0010683063
0.0010701595
0.0010737056
0.0010788608
0.0010846478
0.0010888206
0.001088274
0.0010857587
0.001083558
0.001080629
0.0010746082
0.0010680606
0.0010638083
0.001060815
0.0010591508
0.0010573925
0.0010554928
0.0010544539
0.001054781
0.0010563252
0.0010586027
0.001062315
0.0010614622
0.0010604622
0.0010601734
0.0010578628
0.0010552278
0.0010551832
0.001057073
0.0010605389
0.0010665696
0.0010723314
0.0010780662
0.0010833917
0.0010862083
0.001085966
0.0010806643
0.0010750339
0.0010701324
0.001065667
0.0010619042
0.0010591731
0.0010559114
0.0010543847
0.0010542298
0.0010552949
0.0010580734
0.0010615552
0.001064303
0.0010653331
0.001065592
0.0010647101
0.0010655147
0.0010700526
0.001073244
0.00107369
0.0010724997
0.001070793
0.0010673248
0.0010654746
0.0010650659
0.0010683541
0.0010710574
0.0010732558
0.0010744543
0.0010727728
0.0010716717
0.0010720912
0.0010734744
0.0010773998
0.0010793358
0.0010771755
0.0010728735
0.0010677095
0.0010661898
0.0010655413
0.0010651296
0.0010646624
0

0.0011160295
0.0011165952
0.001114913
0.0011126031
0.001108712
0.0011049756
0.0011021934
0.0011013201
0.0011020869
0.001102832
0.0011057437
0.0011097882
0.0011142653
0.0011204598
0.0011249727
0.0011260805
0.0011252713
0.0011232512
0.0011206664
0.0011168354
0.0011163094
0.0011164206
0.0011156829
0.0011152391
0.0011149906
0.0011144779
0.0011148113
0.0011160055
0.0011164397
0.0011175897
0.0011172071
0.0011173266
0.0011172825
0.0011164146
0.0011136169
0.0011118064
0.0011102043
0.001109538
0.0011113213
0.0011123369
0.001113297
0.0011147831
0.0011149894
0.0011157715
0.0011162972
0.0011161973
0.0011148786
0.001114331
0.0011136995
0.0011143177
0.0011146283
0.0011137333
0.0011120948
0.0011121396
0.0011127031
0.0011119654
0.0011120602
0.0011135212
0.0011149766
0.0011142716
0.001112256
0.0011095271
0.0011081097
0.0011111345
0.0011160526
0.0011207632
0.001123197
0.0011257946
0.0011294263
0.0011309704
0.0011302717
0.0011292754
0.0011286701
0.00112679
0.0011237448
0.0011198939
0.0011177292
0.0011174

0.0011010868
0.0011015532
0.0011015484
0.0010963454
0.0010857959
0.0010753712
0.0010702388
0.0010638672
0.0010572964
0.0010529868
0.0010502429
0.0010485905
0.0010480168
0.001047174
0.0010445688
0.0010428482
0.0010403858
0.0010390512
0.0010360978
0.0010344171
0.0010324408
0.0010319366
0.001031653
0.0010303602
0.0010288996
0.0010270379
0.0010264588
0.0010275044
0.0010288543
0.0010303814
0.0010310977
0.0010322717
0.0010313431
0.0010307711
0.0010285756
0.0010276989
0.0010251985
0.0010237368
0.0010224469
0.0010221252
0.0010241529
0.0010267375
0.0010297017
0.0010304573
0.0010307117
0.0010327352
0.0010346965
0.0010362149
0.0010379858
0.0010413645
0.0010445986
0.0010456215
0.001048091
0.0010496352
0.0010490958
0.0010481868
0.0010463034
0.0010426845
0.0010363194
0.0010323822
0.0010300062
0.0010270576
0.0010261105
0.0010268283
0.0010269317
0.0010308608
0.0010323938
0.0010331217
0.001035401
0.0010372216
0.0010391385
0.0010406526
0.001040128
0.0010383294
0.0010373872
0.0010382931
0.0010374505
0.00

0.0009223629
0.0009267522
0.0009292717
0.0009292222
0.0009288277
0.00092609436
0.0009244813
0.00092355156
0.0009239121
0.0009224574
0.0009209956
0.0009203264
0.0009183145
0.0009168812
0.0009169716
0.0009175148
0.0009177306
0.0009205628
0.00092063204
0.0009221323
0.00092496007
0.00092619663
0.00092653406
0.0009259429
0.000924979
0.00092415186
0.00092461414
0.00092419854
0.0009253435
0.00092878257
0.00093456404
0.0009399077
0.00094282115
0.0009451937
0.0009458418
0.0009471403
0.00094805897
0.0009500825
0.0009521416
0.00095302623
0.0009534377
0.000954609
0.0009545024
0.00095378474
0.00095052284
0.00094862573
0.00095041865
0.0009512593
0.0009503787
0.00094865984
0.0009454377
0.0009424946
0.00093698583
0.00093376916
0.00093053107
0.0009302258
0.0009283293
0.00092593464
0.00092341367
0.00091975625
0.0009142972
0.0009088256
0.00090513664
0.0009020764
0.0009013956
0.00090073573
0.00090203306
0.00090491516
0.00090767327
0.00091080565
0.00091321836
0.0009142186
0.000915352
0.00091809867
0.000919